In [30]:
import numpy as np
import cv2

def object_area(org_image):
  obj_area=0
  kernel = np.ones((5,5),np.uint8)
  gray = cv2.cvtColor(org_image, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0 ,255, cv2.THRESH_OTSU)[1]
  thresh = cv2.erode(thresh,kernel,iterations=5)
  thresh = cv2.dilate(thresh,kernel,iterations=5)
  for i in range(thresh.shape[0]):
    for j in range(thresh.shape[1]):
      if gray[i][j]!=0:
        obj_area+=1#counting the whole object area

  return obj_area

def ostu_cc(org_image):
  #get gray
  gray = cv2.cvtColor(org_image, cv2.COLOR_BGR2GRAY)
  #get ostu
  kernel = np.ones((5,5),np.uint8)
  thresh = cv2.threshold(gray, 0 ,255, cv2.THRESH_OTSU)[1]
  thresh = cv2.erode(thresh,kernel,iterations=5)
  thresh = cv2.dilate(thresh,kernel,iterations=5)
  #get background
  bg = cv2.threshold(gray, 0 ,255, cv2.THRESH_BINARY)[1]
  kernel = np.ones((5,5),np.uint8)
  bg=cv2.erode(bg,kernel,iterations=35)
  #take off background
  obj_area=0
  defects_area=0
  new=[]
  for i in range(thresh.shape[0]):
    ar=[]
    for j in range(thresh.shape[1]):
      if bg[i][j]==0:# if this pixel is black
        ar.append(255)# make the same position in new image is white
      else:
        ar.append(thresh[i][j])# or append the original pixel
      if ar[j]==0:
        defects_area+=1#counting the black pixels
      if gray[i][j]!=0:
        obj_area+=1#counting the whole object area
    new.append(ar)
  new=np.array(new,dtype=np.uint8) # the numpy default dtype is int64, it can't be used in cvtColor
  #reverse b/w
  new_convert=[]
  for i in range(new.shape[0]):
    array=[]
    for j in range(new.shape[1]):
      if new[i][j] == 255:
        array.append(0)
      else:
        array.append(255)
    new_convert.append(array)
  new_convert=np.array(new_convert,dtype=np.uint8)
  #convert defects with colors
  num,labels=cv2.connectedComponents(new_convert)
  label_hue = np.uint8(179*labels/np.max(labels))
  blank_ch = 255*np.ones_like(label_hue)
  labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
  labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
  labeled_img[label_hue==0] = 0 
  #detect different colors area
  defects_gray=cv2.cvtColor(labeled_img,cv2.COLOR_BGR2GRAY)
  counts=[]
  for i in range(0,255):
    counts.append(0)
  for i in defects_gray:
    for j in i:
      if j!=0:
        counts[int(j)]+=1
  for i in range(len(counts)):
    counts[i]=counts[i]/obj_area
  rates=sorted(counts,reverse=True)
  return rates

from skimage.feature import hog
def hog_rates(hog_image,obj_size):
  counts=[]
  for i in range(0,255):
    counts.append(0)
  for i in hog_image:
    for j in i:
      if j!=0:
        counts[int(j)]+=1
  for i in range(len(counts)):
    counts[i]=counts[i]/obj_size
  return counts

def RGB_rates(RGB):
  cnts=[]
  for i in range(86):
    cnts.append(0)
  for i in RGB:
    for j in i:
      cnts[int(j/3)]+=1
  cnts[0]-=RGB.shape[0]*RGB.shape[1]-obj_size
  for i in range(len(cnts)):
    cnts[i]=cnts[i]/obj_size
  return cnts

def list_features():
  features=[]
  features.append("id,")
  for i in range(255):
    features.append("df-"+str(i)+",")
  for i in range(256):
    features.append("hog-"+str(i)+",")
  for i in range(86):
    features.append("R-"+str(i)+",")
  for i in range(86):
    features.append("G-"+str(i)+",")
  for i in range(86):
    features.append("B-"+str(i)+",")
  features.append("target")
  return features

In [33]:
import os

folders=[]
folder_path = "‪d:\mango_SVM\dataset\img"

folders.append('D:\mango_SVM\dataset\img\Train\A')
folders.append('D:\mango_SVM\dataset\img\Train\B')
folders.append('D:\mango_SVM\dataset\img\Train\C')
folders.append('D:\mango_SVM\dataset\img\DEV\A')
folders.append('D:\mango_SVM\dataset\img\DEV\B')
folders.append('D:\mango_SVM\dataset\img\DEV\C')

folder_content = os.listdir('d:\mango_SVM\dataset\img')
LF=list_features()
csv_file = open('d:\mango_SVM\mango_2.csv','w')

# write features(first row)
for i in LF:
  csv_file.write(i)
csv_file.write("\n")
# wirte contents
for c in folders:
  folder_content = os.listdir(c)
  for item in folder_content:
    f = c +"/"+ item
    org_image = cv2.imread(f)
    csv_file.write(item[:5]+",")
    obj_size=object_area(org_image)
    for i in ostu_cc(org_image):
      csv_file.write(str(i)+",")
    fd, hog_image = hog(org_image, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(1, 1), visualise=True, multichannel=True)
    for i in hog_rates(hog_image,obj_size):
      csv_file.write(str(i)+",")
    b, g, r = cv2.split(org_image)
    for i in RGB_rates(r):
      csv_file.write(str(i)+",")
    for i in RGB_rates(g):
      csv_file.write(str(i)+",")
    for i in RGB_rates(b):
      csv_file.write(str(i)+",")
    csv_file.write("c[-1]")
    csv_file.write("\n")
csv_file.close()

TypeError: hog() got an unexpected keyword argument 'multichannel'

In [0]:
import pandas
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

mango = pandas.read_csv("/content/drive/My Drive/data/mango_2.csv") 
X = mango[1:-1].values
Y = mango[-1].values
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)
X = StandardScaler().fit_transform(X)
X_train , X_test, y_train, y_test = train_test_split(X,Y)
clf = SVC(C=1.0, kernel='rbf').fit(X_train,y_train)